In [1]:
import numpy as np
import scipy.sparse as sparse
from numpy import cos, sin, exp, pi
from scipy.sparse.linalg import expm, eigs
from qiskit import transpile, QuantumCircuit
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit import Parameter

param_t = Parameter('t')

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    operators = []
    coefficients = {}
    qubits = 0
    
    __matrix = None
    __unitary = None
    __gates = {}
    #__gate_unitaries = {}
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = sparse.csc_matrix((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def pauli_gate(self, pauli):
        """Gate corresponding to the single trotter step with Pauli string pauli"""
        if pauli not in self.__gates: # unitary has not been calculated yet
            ham = PrimitiveOp(SparsePauliOp(pauli, self.coefficients[pauli])) # Hamiltonian
            ut = PauliTrotterEvolution().convert((param_t*ham).exp_i()) # single Trotter step
            self.__gates[pauli] = transpile(ut.to_circuit(), basis_gates=['u','cx']) # turn into circuit with only allowed gates
        return self.__gates[pauli]
    
    def pauli_gate_unitary(self, pauli, t=1):
        """Unitary matrix corresponding to the single trotter step with Pauli string pauli and t=1"""
        if pauli == 'I' * self.qubits: # identity
            return exp(-1j*self.coefficients[pauli]*t)*sparse.eye(self.dim(), format='csc')
        qc = self.pauli_gate(pauli)
        qc = qc.bind_parameters({param_t: t})
        return sparse.csc_matrix(Operator(qc).data, dtype=np.complex128)
    
    def coefficient(self, operator):
        """Coefficient that goes with the Pauli String operator"""
        return self.coefficients[operator]
    
    def error(self, unitary, tol=0):
        """Largest abs eigenvalue of difference between this Hamiltonian and
           unitary input.
           unitary must be of.sparse csc_matrix type.
           Parameter tol sets precision of eigenvalues (0=machine precision)"""
        diff = self.unitary() - unitary
        ev = eigs(diff, 1, which='LM', return_eigenvectors=False)
        return np.abs(ev[0])        
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
    def trotterize(self, steps):
        """Unitary matrix corresponding to steps Trotter steps"""
        utt = sparse.eye(self.dim(), format='csc')
        t = 1/steps
        for o in self.operators:
            utt = utt.dot(self.pauli_gate_unitary(o, t))
        ut = sparse.eye(self.dim(), format='csc')
        for i in range(steps):
            ut = ut.dot(utt)
        return ut
    
    def trotter_error(self, steps, tol=0):
        return self.error(self.trotterize(steps), tol)
        
            
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h

h = hamiltonian_from_file("hamiltonian_ordered.txt")

In [3]:
%%time
# calculate all gates
for o in h.operators:
    h.pauli_gate(o)

CPU times: user 5.07 s, sys: 45.8 ms, total: 5.11 s
Wall time: 5.18 s


In [3]:
%%time
# calculate all unitaries
for o in h.operators:
    h.pauli_gate_unitary(o, 0.1)

CPU times: user 5min 54s, sys: 16 s, total: 6min 10s
Wall time: 1min 26s


In [2]:
%%time
h.trotter_error(10)

CPU times: user 7min 24s, sys: 20.1 s, total: 7min 44s
Wall time: 2min 20s


0.00730183250120102

In [3]:
%%time
h.trotter_error(5)

CPU times: user 6min 38s, sys: 15.9 s, total: 6min 54s
Wall time: 1min 42s


0.014610994056274864

In [4]:
%%time
h.trotter_error(2)

CPU times: user 6min 31s, sys: 16.5 s, total: 6min 48s
Wall time: 1min 33s


0.03664041840915357

In [5]:
%%time
h.trotter_error(1)

CPU times: user 6min 20s, sys: 16 s, total: 6min 36s
Wall time: 1min 28s


0.07406384276824311